In [1]:
import rospy
import json
import rospkg
from os.path import join
from itertools import permutations
from numpy import mean
from numpy import std
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches
from itertools import combinations
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats.mstats import kruskalwallis
from scipy.stats import sem

In [2]:
res_folder = 'results_pre_study'

In [3]:
res_folder = join('results', 'user_study')

In [4]:
def plot_results(data, savefile):
    width = 0.25
    ind = np.arange(12) + 1

    # extract data
    mean = {}
    ste = {}
    for c in cond_list:
        mean[c] = []
        ste[c] = []
        for i in ind:
            mean[c].append(data[c]['Q' + str(i)]['mean'])
            ste[c].append(data[c]['Q' + str(i)]['sem'])

    fig = plt.figure(facecolor="white")
    ax = fig.add_subplot(111)
    plt.rcParams['font.size'] = 20

    rects1 = ax.bar(ind - 3 * width / 2, mean['fixed'], width, color='tomato', label='fixed')
    (line, caps, _) = plt.errorbar(ind - width, mean['fixed'], ste['fixed'], capsize=3, elinewidth=2,  ecolor='k')

    line.remove()
    for cap in caps:
        cap.set_color('k')
        cap.set_markeredgewidth(3)

    rects2 = ax.bar(ind - width / 2, mean['relative'], width, color='darkseagreen', label='relative')
    (line, caps, _) = plt.errorbar(ind, mean['relative'], ste['relative'], capsize=3, elinewidth=2,  ecolor='k')
    
    line.remove()
    for cap in caps:
        cap.set_color('k')
        cap.set_markeredgewidth(3)

    rects3 = ax.bar(ind + width / 2, mean['reba'], width, color='cornflowerblue', label='optimized')
    (line, caps, _) = plt.errorbar(ind + width, mean['reba'], ste['reba'], capsize=3, elinewidth=2,  ecolor='k')

    line.remove()
    for cap in caps:
        cap.set_color('k')
        cap.set_markeredgewidth(3)

    plt.plot((0, 13), (0, 0), 'k--',lw=2)
    
    legend = ax.legend(loc='lower left')
    # Set the fontsize
    for label in legend.get_texts():
        label.set_fontsize('small')

    for label in legend.get_lines():
        label.set_linewidth(1.5)  # the legend line width

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Average Score')
    ax.set_ylim([-6, 6])
    y_tick = np.arange(13) - 6
    y_label = [str(x) for x in y_tick]
    plt.yticks(y_tick, y_label, rotation='vertical')
    
    labels = []
    for i in ind:
        labels.append('Q' + str(i))
    plt.xticks(ind, labels, rotation='vertical')
    ax.set_xlim([0, 13])

    # save svg file
    fig.set_size_inches(12.8, 10.24)
    plt.savefig(savefile, dpi=100, facecolor=fig.get_facecolor(), transparent=False)
    plt.close()

In [5]:
def test_differrences(data):
    t_test_res = {}
    min_q = min([len(data[c]['questions'][0]) for c in cond_list])
    min_gr = min([len(data[c]['gr0']) for c in cond_list])
    min_sum = min([len(data[c]['sum']) for c in cond_list])
    #min_q = -1
    #min_gr = -1
    #min_sum = -1
    for comb in comb_list:
        t_test_res[comb[0] + '_' + comb[1]] = {}
        t_test_res[comb[0] + '_' + comb[1]]['p_values'] = []
        for i in range(len(data[comb[0]]['questions'])):
            # remove size when number of sample is equal
            #t, p = ttest_ind(data[comb[0]]['questions'][i][:min_q], data[comb[1]]['questions'][i][:min_q])
            t, p = mannwhitneyu(data[comb[0]]['questions'][i][:min_q], data[comb[1]]['questions'][i][:min_q])
            #t, p = kruskalwallis(data[comb[0]]['questions'][i][:min_q], data[comb[1]]['questions'][i][:min_q])
            t_test_res[comb[0] + '_' + comb[1]]['p_values'].append(p)
        for i in range(3):
            #t, p = ttest_ind(data[comb[0]]['gr' + str(i)][:min_gr], data[comb[1]]['gr' + str(i)][:min_gr])
            t, p = mannwhitneyu(data[comb[0]]['gr' + str(i)][:min_gr], data[comb[1]]['gr' + str(i)][:min_gr])
            #t, p = kruskalwallis(data[comb[0]]['gr' + str(i)][:min_gr], data[comb[1]]['gr' + str(i)][:min_gr])
            t_test_res[comb[0] + '_' + comb[1]]['gr' + str(i)] = p
        #t, p = ttest_ind(data[comb[0]]['sum'][:min_sum], data[comb[1]]['sum'][:min_sum])
        t, p = mannwhitneyu(data[comb[0]]['sum'][:min_sum], data[comb[1]]['sum'][:min_sum])
        #t, p = kruskalwallis(data[comb[0]]['sum'][:min_sum], data[comb[1]]['sum'][:min_sum])
        t_test_res[comb[0] + '_' + comb[1]]['sum'] = p
    return t_test_res

In [6]:
def test_cross_differrences(data):
    t_test_res = {}
    for comb in comb_list:
        t_test_res[comb[0] + '_' + comb[1]] = {}
        t_test_res[comb[0] + '_' + comb[1]]['p_values'] = [[], []]
        for i in range(len(data[comb[0]]['questions'])):
            # remove size when number of sample is equal
            min_q = min(len(data[comb[0] + '_' + comb[1]]['questions'][0][i]),
                        len(data[comb[1] + '_' + comb[0]]['questions'][0][i]))
            t, p = mannwhitneyu(data[comb[0] + '_' + comb[1]]['questions'][0][i][:min_q],
                                data[comb[1] + '_' + comb[0]]['questions'][0][i][:min_q])
            t_test_res[comb[0] + '_' + comb[1]]['p_values'][0].append(p)
            # -------------------------
            min_q = min(len(data[comb[0] + '_' + comb[1]]['questions'][1][i]),
                        len(data[comb[1] + '_' + comb[0]]['questions'][1][i]))
            t, p = mannwhitneyu(data[comb[0] + '_' + comb[1]]['questions'][1][i][:min_q],
                                data[comb[1] + '_' + comb[0]]['questions'][1][i][:min_q])
            t_test_res[comb[0] + '_' + comb[1]]['p_values'][1].append(p)
        for i in range(3):
            min_gr = min(len(data[comb[0] + '_' + comb[1]]['gr' + str(i)][0]),
                         len(data[comb[1] + '_' + comb[0]]['gr' + str(i)][0]))
            t, pA = mannwhitneyu(data[comb[0] + '_' + comb[1]]['gr' + str(i)][0][:min_gr],
                                data[comb[1] + '_' + comb[0]]['gr' + str(i)][0][:min_gr])
            #-----------------------------
            min_gr = min(len(data[comb[0] + '_' + comb[1]]['gr' + str(i)][1]),
                        len(data[comb[1] + '_' + comb[0]]['gr' + str(i)][1]))
            t, pB = mannwhitneyu(data[comb[0] + '_' + comb[1]]['gr' + str(i)][1][:min_gr],
                                 data[comb[1] + '_' + comb[0]]['gr' + str(i)][1][:min_gr])
            t_test_res[comb[0] + '_' + comb[1]]['gr' + str(i)] = [pA, pB]
        min_sum = min(len(data[comb[0] + '_' + comb[1]]['sum'][0]),
                      len(data[comb[1] + '_' + comb[0]]['sum'][0]))
        t, pA = mannwhitneyu(data[comb[0] + '_' + comb[1]]['sum'][0][:min_sum],
                             data[comb[1] + '_' + comb[0]]['sum'][0][:min_sum])
        #------------------------------
        min_sum = min(len(data[comb[0] + '_' + comb[1]]['sum'][1]),
                      len(data[comb[1] + '_' + comb[0]]['sum'][1]))
        t, pB = mannwhitneyu(data[comb[0] + '_' + comb[1]]['sum'][1][:min_sum],
                             data[comb[1] + '_' + comb[0]]['sum'][1][:min_sum])
        t_test_res[comb[0] + '_' + comb[1]]['sum'] = [pA, pB]
    return t_test_res

In [7]:
rospy.init_node('survey_analysis')

In [8]:
rospack = rospkg.RosPack()
res_dir = join(rospack.get_path('thr_xp_human_comfort'), res_folder)
filename = join(res_dir, 'survey.json')
with open(filename) as datafile:
    survey_data = json.load(datafile)

In [9]:
cond_list = ['reba', 'relative', 'fixed']
comb_list = list(combinations(cond_list, 2))
perm_list = list(permutations(cond_list, 2))

In [10]:
results = {}
results['order'] = {}
results['order']['fixed'] = 0
results['order']['reba'] = 0
results['order']['relative'] = 0

results['questions'] = {}
results['questions']['A'] = {}
results['questions']['B'] = {}
for c in cond_list:
    results['questions'][c] = {}
    results['questions']['A'][c] = {}
    results['questions']['B'][c] = {}
    for i in range(1, 13):
        results['questions'][c]['Q' + str(i)] = {}
        results['questions'][c]['Q' + str(i)]['data'] = []
        results['questions']['A'][c]['Q' + str(i)] = {}
        results['questions']['A'][c]['Q' + str(i)]['data'] = []
        results['questions']['B'][c]['Q' + str(i)] = {}
        results['questions']['B'][c]['Q' + str(i)]['data'] = []
        
for p in perm_list:
    results['questions'][p[0] + '_' + p[1]] = [[],[]]

# add dict to check number of people in each conditions
results['conditions'] = {}
for c in cond_list:
    results['conditions'][c] = 0
for p in perm_list:
    results['conditions'][p[0] + '-' + p[1]] = 0.

In [11]:
# get total number of user
results['nb_users'] = len(survey_data)
results['ages'] = []
# go through all results
for user_data in survey_data:
    user_id = str(user_data['ID'])
    # open config file of the user
    filename = join(res_dir, user_id, 'user_config.json')
    with open(filename) as datafile:
        user_config = json.load(datafile)
    user_cond = user_config['conditions']

    # collect age
    results['ages'].append(int(user_data['Age']))

    if user_config['laterality'] == 'right':
        # add conditions
        for c in user_cond:
            results['conditions'][c] += 1
        results['conditions'][user_cond[0] + '-' + user_cond[1]] += 1

        # get preference order
        if user_data['O1[1]'] == 'A':
            results['order'][user_cond[0]] += 1
        else:
            results['order'][user_cond[1]] += 1

        code_list = ['A', 'B']
        # get survey responses
        for i in range(2):
            for j in range(1, 13):
                q_score = (-1)**(j-1) * user_data['Q' + code_list[i] + '1[SQ' + code_list[i] + str(j) + ']']
                results['questions'][user_cond[i]]['Q' + str(j)]['data'].append(q_score)
                results['questions'][code_list[i]][user_cond[i]]['Q' + str(j)]['data'].append(q_score)
                results['questions'][user_cond[0] + '_' + user_cond[1]][i].append(q_score)

In [12]:
nb_user_cond = min([results['conditions']['reba'],
               results['conditions']['relative'],
               results['conditions']['fixed']])

In [13]:
print nb_user_cond, results['nb_users']

23 39


In [14]:
print 'age: ' + str(mean(results['ages'])) + ' +- ' + str(std(results['ages']))
print 'oldest: ' + str(max(results['ages']))
print 'youngest: ' + str(min(results['ages']))

age: 35.1025641026 +- 11.2632710824
oldest: 61
youngest: 21


In [15]:
# calculate global satisfaction and check statistical differences
data_test = {}
data_test['A'] = {}
data_test['B'] = {}
for c in cond_list:
    data_test[c] = {}
    data_test[c]['questions'] = []
    data_test[c]['sum'] = []
    data_test['A'][c] = {}
    data_test['A'][c]['questions'] = []
    data_test['A'][c]['sum'] = []
    data_test['B'][c] = {}
    data_test['B'][c]['questions'] = []
    data_test['B'][c]['sum'] = []
    for i in range(3):
        data_test[c]['gr' + str(i)] = []
        data_test['A'][c]['gr' + str(i)] = []
        data_test['B'][c]['gr' + str(i)] = []
    results['questions'][c]['satisfaction'] = 0.
    for i in range(1, 13):
        q = results['questions'][c]['Q' + str(i)]
        qA = results['questions']['A'][c]['Q' + str(i)]
        qB = results['questions']['B'][c]['Q' + str(i)]
        data_test[c]['questions'].append(q['data'])
        data_test[c]['sum'] += q['data']
        data_test['A'][c]['questions'].append(qA['data'])
        data_test['A'][c]['sum'] += qA['data']
        data_test['B'][c]['questions'].append(qB['data'])
        data_test['B'][c]['sum'] += qB['data']
        if i < 5:
            data_test[c]['gr0'] += q['data']
            data_test['A'][c]['gr0'] += qA['data']
            data_test['B'][c]['gr0'] += qB['data']
        elif i < 9:
            data_test[c]['gr1'] += q['data']
            data_test['A'][c]['gr1'] += qA['data']
            data_test['B'][c]['gr1'] += qB['data']
        else:
            data_test[c]['gr2'] += q['data']
            data_test['A'][c]['gr2'] += qA['data']
            data_test['B'][c]['gr2'] += qB['data']
        # calculate mean of results
        q['mean'] = mean(q['data'])
        q['sem'] = sem(q['data'])
        qA['mean'] = mean(qA['data'])
        qA['sem'] = sem(qA['data'])
        qB['mean'] = mean(qB['data'])
        qB['sem'] = sem(qB['data'])
        results['questions'][c]['satisfaction'] += q['mean']
    results['questions'][c]['satisfaction'] /= 12
for p in perm_list:
    data_test[p[0] + '_' + p[1]] = {}
    data_test[p[0] + '_' + p[1]]['questions'] = [data_test['A'][p[0]]['questions'],
                                                 data_test['B'][p[1]]['questions']]
    data_test[p[0] + '_' + p[1]]['sum'] = [data_test['A'][p[0]]['sum'], data_test['B'][p[1]]['sum']]
    for i in range(3):
        data_test[p[0] + '_' + p[1]]['gr' + str(i)] = [data_test['A'][p[0]]['gr' + str(i)],
                                                       data_test['B'][p[1]]['gr' + str(i)]]
    

In [16]:
plot_results(results['questions'], join(res_dir, 'questions_results.svg'))
plot_results(results['questions']['A'], join(res_dir, 'questions_results_A.svg'))
plot_results(results['questions']['B'], join(res_dir, 'questions_results_B.svg'))

In [17]:
t_test_res = test_differrences(data_test)
t_test_A = test_differrences(data_test['A'])
t_test_B = test_differrences(data_test['B'])
for c in cond_list:
    t, p = mannwhitneyu(data_test['A'][c]['sum'], data_test['A'][c]['sum'])
    print c + ': ' + str(p)
    for i in range(3):
        t, p = mannwhitneyu(data_test['A'][c]['gr' + str(i)], data_test['A'][c]['gr' + str(i)])
        print c + ' gr' + str(i) + ': ' + str(p)
    print '----------------------------------'

reba: 0.49967350794
reba gr0: 0.498300237579
reba gr1: 0.498301609201
reba gr2: 0.498308602399
----------------------------------
relative: 0.499715492272
relative gr0: 0.49852590263
relative gr1: 0.498515211735
relative gr2: 0.498520298247
----------------------------------
fixed: 0.499773013442
fixed gr0: 0.498818550941
fixed gr1: 0.498824403724
fixed gr2: 0.498818425462
----------------------------------


In [18]:
test_data = t_test_res

In [19]:
for key, value in test_data.iteritems():
    p_values = value['p_values']
    value['not_independant'] = {}
    value['not_independant']['question'] = []
    value['not_independant']['p_value'] = []
    for i in range(len(p_values)):
        if p_values[i] > 0.001:
            value['not_independant']['question'].append(i + 1)
            value['not_independant']['p_value'].append(p_values[i])
for key, value in test_data.iteritems():
    print key + ': ' + str(value['not_independant']['question'])
    print value['not_independant']['p_value']

reba_fixed: [6]
[0.23232462077870863]
reba_relative: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[0.069641338926747481, 0.015402324406757597, 0.12679501395498521, 0.17398706755999521, 0.005301307919102825, 0.19758879736815216, 0.18508800207719461, 0.026518985516295258, 0.045214130380534977, 0.05576594875391977, 0.013928159724883182, 0.044480287384171223]
relative_fixed: [4, 6]
[0.018947877226769057, 0.078267838406499396]


In [20]:
for key,value in test_data.iteritems():
    print key + ': ' + str(value['sum'])
    for i in range(3):
        print key + ' gr' + str(i) + ': ' + str(value['gr' + str(i)])
    print '----------------------------------'

reba_fixed: 3.63111813167e-12
reba_fixed gr0: 2.27727536742e-05
reba_fixed gr1: 0.00313515449544
reba_fixed gr2: 3.98924383062e-05
----------------------------------
reba_relative: 0.061729240763
reba_relative gr0: 0.190049382458
reba_relative gr1: 0.331489873203
reba_relative gr2: 0.0910618044584
----------------------------------
relative_fixed: 2.3675328526e-10
relative_fixed gr0: 0.000248222090584
relative_fixed gr1: 0.0021715186514
relative_fixed gr2: 0.000396623262924
----------------------------------


In [21]:
# test differences between permutations
t, p = mannwhitneyu(results['questions']['reba_fixed'][0], results['questions']['reba_relative'][0])
print 'optimized_fixed & optimized_relative: ' + str(p)
t, p = mannwhitneyu(results['questions']['fixed_reba'][1], results['questions']['relative_reba'][1])
print 'fixed_optimized & relative_optimized: ' + str(p)
print '----------------------'
t, p = mannwhitneyu(results['questions']['relative_fixed'][0], results['questions']['relative_reba'][0])
print 'relative_fixed & relative_optimized: ' + str(p)
t, p = mannwhitneyu(results['questions']['fixed_relative'][1], results['questions']['reba_relative'][1])
print 'fixed_relative & optimized_relative: ' + str(p)
print '----------------------'
t, p = mannwhitneyu(results['questions']['fixed_reba'][0], results['questions']['fixed_relative'][0])
print 'fixed_optimized & fixed_relative: ' + str(p)
t, p = mannwhitneyu(results['questions']['reba_fixed'][1], results['questions']['relative_fixed'][1])
print 'relative_fixed & optimized_fixed: ' + str(p)

print '-------------------'
print '-------------------'

reba_fixed =  results['questions']['reba_fixed'][0] + results['questions']['fixed_reba'][1]
reba_relative = results['questions']['reba_relative'][0] + results['questions']['relative_reba'][1]
relative_fixed =  results['questions']['relative_fixed'][0] + results['questions']['fixed_relative'][1]
relative_reba = results['questions']['relative_reba'][0] + results['questions']['reba_relative'][1]
fixed_relative =  results['questions']['fixed_relative'][0] + results['questions']['relative_fixed'][1]
fixed_reba = results['questions']['fixed_reba'][0] + results['questions']['reba_fixed'][1]

t, p = mannwhitneyu(reba_fixed, reba_relative)
print 'optimized: ' + str(p)
t, p = mannwhitneyu(relative_fixed, relative_reba)
print 'relative: ' + str(p)
t, p = mannwhitneyu(fixed_reba, fixed_relative)
print 'fixed: ' + str(p)

print '-------------------'
print '-------------------'

t, p = mannwhitneyu(reba_relative, reba_fixed)
print 'reba in both: ' + str(p)
t, p = mannwhitneyu(fixed_reba, relative_reba)
print 'fixed & relative: ' + str(p)

optimized_fixed & optimized_relative: 0.496297916098
fixed_optimized & relative_optimized: 0.0976485399098
----------------------
relative_fixed & relative_optimized: 0.0255727862168
fixed_relative & optimized_relative: 5.86094949602e-05
----------------------
fixed_optimized & fixed_relative: 0.270726291298
relative_fixed & optimized_fixed: 0.05748020193
-------------------
-------------------
optimized: 0.148298584696
relative: 2.31410444113e-05
fixed: 0.24967314074
-------------------
-------------------
reba in both: 0.148298584696
fixed & relative: 0.00159296762873


In [22]:
cross_test_res = test_cross_differrences(data_test)

In [23]:
for j in range(2):
    for key, value in cross_test_res.iteritems():
        p_values = value['p_values']
        value['not_independant'] = {}
        value['not_independant']['question'] = []
        value['not_independant']['p_value'] = []
        for i in range(len(p_values[j])):
            if p_values[j][i] > 0.1:
                value['not_independant']['question'].append(i + 1)
                value['not_independant']['p_value'].append(p_values[j][i])
    for key, value in cross_test_res.iteritems():
        print key + ': ' + str(value['not_independant']['question'])
        print value['not_independant']['p_value']
    print '-----------'

reba_fixed: [6]
[0.41179800009869477]
reba_relative: [1, 3, 4, 6, 7, 8, 10, 12]
[0.16459056160102736, 0.15056469664927774, 0.20882975597077791, 0.11573396966706312, 0.16278015552524422, 0.28077267485750534, 0.25783326769064041, 0.17677356728893401]
relative_fixed: [6]
[0.19067010984298954]
-----------
reba_fixed: [6]
[0.11499520144544029]
reba_relative: [1, 3, 4, 6, 7, 9, 11, 12]
[0.12749446090220801, 0.27336419874770568, 0.30591596750178396, 0.5, 0.42257851721077688, 0.32556406435780361, 0.3657907488170995, 0.11641721422683032]
relative_fixed: [6]
[0.13460646457171777]
-----------


In [25]:
for key,value in cross_test_res.iteritems():
    print key + ': ' + str(value['sum'])
    for i in range(3):
        print key + ' gr' + str(i) + ': ' + str(value['gr' + str(i)])
    print '----------------------------------'

reba_fixed: [3.5051160858222631e-06, 1.3730276461676855e-08]
reba_fixed gr0: [0.014516138894293408, 0.00027334792957232407]
reba_fixed gr1: [0.12205625558650468, 0.0026465366916853953]
reba_fixed gr2: [0.011109624554778465, 0.00026047923429939042]
----------------------------------
reba_relative: [0.089207089786599691, 0.19261741040153768]
reba_relative gr0: [0.21651274316385721, 0.31448956315520732]
reba_relative gr1: [0.45602214910209371, 0.31767474917693517]
reba_relative gr2: [0.10922645448031215, 0.27407327928522718]
----------------------------------
relative_fixed: [5.362969897884047e-05, 3.0102575336362049e-07]
relative_fixed gr0: [0.026332203908336359, 0.0022322643036002247]
relative_fixed gr1: [0.088621050405935098, 0.0041346580220441506]
relative_fixed gr2: [0.029014340537799117, 0.0017993026490930166]
----------------------------------
